Facebook Data Crawling

After the data is collected, we need to process the data to make it more readable and easier to analyze. We can do some simple visualization to see the data distribution and the relationship between the data. We can also do some simple data cleaning to remove the data that is not needed. In this notebook, we will do some simple data processing on the data we collected from Facebook.

In [1]:
%pip install matplotlib pandas numpy seaborn wordcloud

                                              0.0/294.6 kB ? eta -:--:--
                                              0.0/294.6 kB ? eta -:--:--
     ----                                     30.7/294.6 kB ? eta -:--:--
     ----                                     30.7/294.6 kB ? eta -:--:--
     -----                                 41.0/294.6 kB 281.8 kB/s eta 0:00:01
     -----------                           92.2/294.6 kB 525.1 kB/s eta 0:00:01
     ---------------                      122.9/294.6 kB 658.7 kB/s eta 0:00:01
     -----------------                    143.4/294.6 kB 610.6 kB/s eta 0:00:01
     -----------------------              194.6/294.6 kB 737.3 kB/s eta 0:00:01
     ------------------------------------ 294.6/294.6 kB 960.6 kB/s eta 0:00:00
                                              0.0/151.4 kB ? eta -:--:--
     ---------------------                   81.9/151.4 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 151.4/151.4 kB 2.2 MB/s eta 0


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json

In [ ]:
raw_df = pd.read_csv('Data/HonkaiStarRail.VN.csv')

In [ ]:
raw_df.info()

In [ ]:
first_row = raw_df.iloc[0]
reactions = dict(eval(first_row['reactions']))
reactions["shares"] = first_row['shares']
reactions["comments"] = first_row['comments']

In [ ]:
reactions

Reactions analysis

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(15,5)})
sns.set(font_scale=1.5)

# plot the reactions
fig, ax = plt.subplots()
ax = sns.barplot(x=list(reactions.keys()), y=list(reactions.values()))
ax.set_title("Reactions to HonkaiStarRail's Facebook Posts")
ax.set_ylabel("Number of Reactions")
ax.set_xlabel("Type of Reaction")
plt.xticks(rotation=-45)
plt.show()

Read all posts

In [ ]:
df_text = post_df[['post_id','post_text']]
text    = str(df_text['post_text'])

df_text

In [ ]:
from wordcloud import WordCloud, STOPWORDS

wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=300,
                      width=2000, height=1200
                         ).generate(text)
plt.figure(figsize=(40,20))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# Find the earliest and latest post
earliest_post_time = df_filtered['time'].min()
latest_post_time = df_filtered['time'].max()

In [ ]:
print("Thời gian bài đăng sớm nhất:", earliest_post_time)
print("Thời gian bài đăng mới nhất:", latest_post_time)

In [ ]:
# Find the total interactions of all posts
df_filtered["total_interactions"] = df_filtered["reaction_count"] + df_filtered["comments"] + df_filtered["shares"]

print("Tổng số lượt tương tác:", df_filtered["total_interactions"].sum())
print("Lượt tương tác trung bình:", int(df_filtered["total_interactions"].mean()))

In [ ]:
# Find the post which has the highest number of interactions
most_interacted_post = df_filtered.loc[df_filtered["total_interactions"].idxmax()]

print("Bài viết có lượng tương tác lớn nhất:")
print(most_interacted_post["post_text"])
print("Lượt thích:", most_interacted_post["reaction_count"])
print("Bình luận:", most_interacted_post["comments"])
print("Chia sẻ:", most_interacted_post["shares"])
print("Thời gian:",most_interacted_post["time"])

In [ ]:
# Tạo một cột mới 'post_hour' để nhóm dữ liệu theo giờ
df_filtered['post_hour'] = (df_filtered['time'].dt.hour ) 

In [ ]:
df_filtered['post_hour']

In [ ]:
# Nhóm dữ liệu theo giờ và đếm số lượng bài đăng trong mỗi khoảng thời gian
post_per_hour = df_filtered.groupby('post_hour').size().reset_index(name='post_count')

In [ ]:
# lượng bài đăng theo thời gian
plt.figure(figsize=(12, 6))
plt.bar(post_per_hour['post_hour'], post_per_hour['post_count'])
plt.title('Sự Chênh Lệch Số Lượng Bài Đăng Theo Giờ')
plt.xlabel('Giờ trong Ngày')
plt.ylabel('Số Lượng Bài Đăng')
plt.xticks(range(24))
plt.show()

In [ ]:
# Nhóm dữ liệu theo giờ và đếm số lượng tương tác trong mỗi khoảng thời gian
interaction_per_hour = df_filtered.groupby('post_hour')["total_interactions"].sum()

In [ ]:
interaction_per_hour.plot(kind='bar', xlabel='Giờ trong Ngày', ylabel='Số Lượng tương tác')

In [ ]:
# Lấy 3 thời điểm có số lượng bài đăng lớn nhất
top_3_hours = post_per_hour.nlargest(3, 'post_count')

print("3 thời điểm có nhiều bài đăng nhất:")
for index, row in top_3_hours.iterrows():
    print(f"Giờ {row['post_hour']}:00 với {row['post_count']} bài đăng")

keyword

In [ ]:
df_text = df_filtered[['post_id','post_text']]
text    = str(df_text['post_text'])

df_text

In [ ]:
from wordcloud import WordCloud, STOPWORDS

wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=300,
                      width=2000, height=1200
                         ).generate(text)
plt.figure(figsize=(40,20))
plt.clf()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()